## Task
This [task](https://www.reddit.com/r/DataVizRequests/comments/7b7iz3/i_would_like_for_someone_to_visualize_this/) was requested by [/u/adubyouu](www.reddit.com/u/adubyouu) to gather and clean some data regarding community impact scores and opportunity scores in census tracts and zip codes in Pennsylvania. 

He gave out this [data source](http://www.phfa.org/forms/multifamily_application_guidelines/presentation/2018_community_impact_opportunity_scores.pdf) which had 2 main chunks of data:
1. Community Impact Scores - by Census Tracts Code (short form)
2. Opportunity Scores - by Zip Code

## Approach
My order of approach was to convert the Census Tracts into Zip Codes, then plot the data. Simple right? It turns out that the representation for Census tract has some quirks, also that there is some trickyness to converting census tracts to zip codes.

### Zip Codes
Zip Codes actually change pretty often and are determined by the post office. But I believe they are 'stable enough' for most purposes. I had a LOT of trouble trying to map zip codes in python. It would either take a gigantic download (and I was getting issues with data throughput on my machine and jupyter) of 1.2gb, or extra processing and hopes that breaking it down by state would make it magically work. There were some [solutions](http://www.christianpeccei.com/zipmap/), but they were really old and the libraries werent supported anymore. I found that it was acutally a [very easy task](https://www.arilamstein.com/open-source/choroplethrzip/creating-zip-code-choropleths-choroplethrzip/) with R.

### Census Tracts
If you haven't worked with census tracts before, they are given by the US govt, and are made every census. They have special codes like `47009011603`. Breaking this code down the first 2 digits **`47`** represent the state - Pennsylvania. The next 3 digits **`009`** represent the county - Bedford County. Lastly the rest of the digits **`011603`** represent the census tract. This gets a little tricky because there is an implied decimal place, so the tract code is actually **`116.03`**. I found this [resource](https://data.pa.gov/Government-That-Works/FIPS-Codes-for-PA-Counties/44ch-j9ei) online which let me know which county was which. 

Let's get started before you fall asleep though ;)

First I import and clean the source data. I actually cheated and used an online source to convert the pdf source into csv... Python can do this with tabula, but it wasnt working at all.

In [1]:
import pandas as pd
import folium
import os
import numpy as np

project_dir = os.path.join(os.getcwd(), os.pardir)
ci_path = os.path.join(project_dir, 'data', 'community_impact_scores.csv')
os_path = os.path.join(project_dir, 'data', 'opportunity_scores.csv')

op_score = pd.read_csv(os_path)
ci_score = pd.read_csv(ci_path)

In [2]:
del op_score['Unnamed: 3']
del op_score['Unnamed: 4']
del ci_score['Unnamed: 3']
del ci_score['Unnamed: 4']

In [3]:
ci_score['tract_id_proper'] = ci_score['CENSUS TRACT'].str.extract(r'Census Tract (.+),.+, Pennsylvania')
ci_score.head()

C:\Users\DT-Laptop\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,CENSUS TRACT,COUNTY,COMMUNITY IMPACT SCORE,tract_id_proper
0,"Census Tract 301.01, Adams County, Pennsylvania",Adams,2.0,301.01
1,"Census Tract 301.02, Adams County, Pennsylvania",Adams,1.0,301.02
2,"Census Tract 302, Adams County, Pennsylvania",Adams,3.5,302
3,"Census Tract 303, Adams County, Pennsylvania",Adams,2.0,303
4,"Census Tract 304, Adams County, Pennsylvania",Adams,2.0,304


In [4]:
op_score['AREA'] = op_score.AREA.str.extract(r'\((.+)\)')
op_score.head()

C:\Users\DT-Laptop\Anaconda3\envs\ds\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,ZIP CODE,AREA,OPPORTUNITY SCORE
0,15001,"Aliquippa, PA",3
1,15003,"Ambridge, PA",3
2,15004,"Atlasburg, PA",3
3,15005,"Baden, PA",3.5
4,15006,"Bairdford, PA",2.75


The OP found [this](https://www.huduser.gov/portal/datasets/usps_crosswalk.html) correlation between zip codes and census tracts. Notice the long form of the tract code.

In [5]:
tract_zip_correlation = pd.read_excel(os.path.join(project_dir, 'data', 'ZIP_TRACT_092017.xlsx'))

In [6]:
tract_zip_correlation.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37801,47009011603,0.035392,0.041385,0.000000,0.035849
1,37801,47009011200,0.339105,0.448480,0.333333,0.348114
2,37766,47013950800,0.086610,0.062833,0.008021,0.082289
3,37801,47009011604,0.068884,0.010135,0.133333,0.064110
4,601,72001956700,0.668537,0.420792,0.500000,0.649833


I didnt end up actually using this correlation between the county name and the long tract code, but its neat to know how things work. Its also complete for future usage.

In [7]:
state_fips = pd.read_csv(os.path.join(project_dir, 'data', 'FIPS_Codes_for_PA_Counties.csv'))
state_fips.head()

,State Name Abbreviation,State Name,State FIPS Code,County FIPS Code,County Code,Short County Name,County Name,FIPS Class Code
0,PA,Pennsylvania,42,39,20,Crawford,Crawford County,H1
1,PA,Pennsylvania,42,79,40,Luzerne,Luzerne County,H1
2,PA,Pennsylvania,42,77,39,Lehigh,Lehigh County,H1
3,PA,Pennsylvania,42,83,42,McKean,McKean County,H1
4,PA,Pennsylvania,42,89,45,Monroe,Monroe County,H1


I discovered that the correlations were for all the US! So I limited them to just Pennsylvania for efficiency. 

In [8]:
pa_temp = tract_zip_correlation[tract_zip_correlation.tract.astype(str).str[:2] == '42'].reset_index(drop=True)

In [9]:
pa_temp['County FIPS Code'] = pa_temp.tract.astype(str).str[2:5].astype(int)
pa_temp['tract_id'] = pa_temp.tract.astype(str).str[5:]

pa_tracts = pa_temp.merge(state_fips[['County FIPS Code', 'Short County Name']], on='County FIPS Code',)
pa_tracts.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio,County FIPS Code,tract_id,Short County Name
0,19438,42091207500,0.073046,0.056162,0.046729,0.071769,91,207500,Montgomery
1,18936,42091200606,0.866328,0.460237,0.121212,0.648560,91,200606,Montgomery
2,19001,42091201607,0.257411,0.496307,0.233577,0.277914,91,201607,Montgomery
3,19002,42091201201,0.086873,0.002693,0.000000,0.079326,91,201201,Montgomery
4,19002,42091201411,0.036726,0.013465,0.018692,0.034726,91,201411,Montgomery


In [10]:
pa_tracts['tract_id_proper'] = pa_tracts.tract_id.astype(float)/100
pa_tracts['tract_id_proper'] = pa_tracts.tract_id_proper.astype(str).str.replace('\.0$', '')
pa_tracts.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio,County FIPS Code,tract_id,Short County Name,tract_id_proper
0,19438,42091207500,0.073046,0.056162,0.046729,0.071769,91,207500,Montgomery,2075
1,18936,42091200606,0.866328,0.460237,0.121212,0.648560,91,200606,Montgomery,2006.06
2,19001,42091201607,0.257411,0.496307,0.233577,0.277914,91,201607,Montgomery,2016.07
3,19002,42091201201,0.086873,0.002693,0.000000,0.079326,91,201201,Montgomery,2012.01
4,19002,42091201411,0.036726,0.013465,0.018692,0.034726,91,201411,Montgomery,2014.11


Im finally starting to correlate the community impact score and zip code.

In [11]:
ci_adjusted = ci_score.merge(pa_tracts[['zip', 'tot_ratio', 'tract_id_proper']], on='tract_id_proper')
ci_adjusted[ci_adjusted['zip'] == 17550]

,CENSUS TRACT,COUNTY,COMMUNITY IMPACT SCORE,tract_id_proper,zip,tot_ratio
13157,"Census Tract 111, Blair County, Pennsylvania",Blair,2.0,111,17550,1.0
13179,"Census Tract 111, Cambria County, Pennsylvania",Cambria,2.0,111,17550,1.0
13201,"Census Tract 111, Centre County, Pennsylvania",Centre,3.5,111,17550,1.0
13223,"Census Tract 111, Franklin County, Pennsylvania",Franklin,3.0,111,17550,1.0
13245,"Census Tract 111, Lancaster County, Pennsylvania",Lancaster,1.0,111,17550,1.0
13267,"Census Tract 111, Lawrence County, Pennsylvania",Lawrence,2.5,111,17550,1.0
13289,"Census Tract 111, Lycoming County, Pennsylvania",Lycoming,3.0,111,17550,1.0
13311,"Census Tract 111, Northampton County, Pennsylv...",Northampton,5.0,111,17550,1.0
13333,"Census Tract 111, Philadelphia County, Pennsyl...",Philadelphia,5.0,111,17550,1.0


Here is an extremely important step. I use the total ratio (depending on use case this could be `res_ratio` as well) to scale (multiply) the scores to convert the census tracts to zip codes. And then I group by zip and add up the sum of the parts (each ratio\*score). I found that some of the sums of tot_ratios are *greater than 1*. This is a problem, so I treated it like an average and divided by the summed per zip coded tot_ratio. This happend because there is overlap. The normalization fixes the issue when we have a tot_ratio that *sums* to more than 1. 

In [12]:
ci_adjusted = ci_score.merge(pa_tracts[['zip', 'tot_ratio', 'tract_id_proper']], on='tract_id_proper')
ci_adjusted['new_community_score'] = ci_adjusted['COMMUNITY IMPACT SCORE'] * ci_adjusted.tot_ratio
ci_adjusted = ci_adjusted.groupby('zip').sum()
ci_adjusted['normalized_community_score'] = ci_adjusted.new_community_score / ci_adjusted.tot_ratio
ci_adjusted.head()

,COMMUNITY IMPACT SCORE,tot_ratio,new_community_score,normalized_community_score
zip,,,,
15001,30.5,1.0,2.398820,2.398820
15003,21.0,1.0,3.444746,3.444746
15004,2.5,1.0,2.500000,2.500000
15005,11.0,1.0,2.264440,2.264440
15006,1.5,1.0,1.500000,1.500000


Lastly I combine the Community Impact data with the Opportunity data and write it out to a csv.

In [13]:
df = ci_adjusted.merge(op_score[['OPPORTUNITY SCORE', 'ZIP CODE']], left_index=True, right_on='ZIP CODE')
df[df['OPPORTUNITY SCORE'] == '#DIV/0!'] = 0
df['OPPORTUNITY SCORE'] = pd.to_numeric(df['OPPORTUNITY SCORE'])

In [14]:
# df.to_feather(os.path.join(project_dir, 'data', 'ci_and_op_data.feather'))
df.to_csv(os.path.join(project_dir, 'data', 'ci_and_op_data.csv'))

I'm having trouble running R in Jupyter.. but I created the plots in R. The process was done just like [here](https://www.arilamstein.com/open-source/choroplethrzip/creating-zip-code-choropleths-choroplethrzip/).